In [86]:
import numpy as np
import copy
import math
import random
from collections import defaultdict
import itertools
import csv

In [2]:
class Vec:
    def __init__(self,x_pos,y_pos):
        self.x = x_pos
        self.y = y_pos
        
    def __hash__(self):
        return hash((self.x, self.y))
    
    def __eq__(self, other):
        if (isinstance(other, Vec)):
            return self.x == other.x and self.y == other.y
        return False

In [3]:
class AABB:
    def __init__(self,obj_bl,obj_tr):
        self.tr = obj_tr
        self.bl = obj_bl
    def __hash__(self):
        return hash((self.tr, self.bl))
    
    def __eq__(self, other):
        if (isinstance(other, AABB)):
            return self.tr == other.tr and self.bl == other.bl
        return False

In [4]:
def cellSizeCalc(list_AABBs):
    max_x,max_y = 0,0
    for aabb in list_AABBs:
        if (aabb.tr.x > max_x):
            max_x = aabb.tr.x
        if (aabb.tr.y > max_y):
            max_y = aabb.tr.y
    cellSize = float(max_x / len(list_AABBs)) if max_x < max_y else float(max_y / len(list_AABBs))
    return cellSize,max_y,max_x

In [5]:
def check_collision(here,another):
    return not (another.bl.x > here.tr.x or another.tr.x < here.bl.x or another.tr.y < here.bl.y or another.bl.y > here.tr.y)

In [6]:
class Cell:
    def __init__(self):
        self.list_AABBs_idxs = []
    def appendCell(self,AABB_obj_idx):
        self.list_AABBs.append(AABB_obj_idx)
    def removeCell(self,AABB_obj_idx):
        self.list_AABBs.remove(AABB_obj_idx)
    def getObj(self):
        return self.list_AABBs_idxs
    

In [84]:
class Grid:
    def __init__(self,list_AABBs):
        self.list_AABBs = list_AABBs
        self.cellSize,self.height,self.width = cellSizeCalc(list_AABBs)
        self.numXCells = math.ceil(float(self.width / self.cellSize))
        self.numYCells = math.ceil(float(self.height / self.cellSize))
        self.list_cells = [[Cell() for j in range(self.numXCells+1)] for i in range(self.numYCells+1)]
        self.obj_dict = defaultdict(list)
        self.list_ret = []
    def AABBtoCell(self,AABB_obj_idx):
        upper_y = int(self.list_AABBs[AABB_obj_idx].tr.y / self.cellSize) + 1 if self.list_AABBs[AABB_obj_idx].tr.y % self.cellSize == 0 else int(self.list_AABBs[AABB_obj_idx].tr.y / self.cellSize)
        lower_y = int(self.list_AABBs[AABB_obj_idx].bl.y / self.cellSize) + 1 if self.list_AABBs[AABB_obj_idx].bl.y % self.cellSize == 0 else int(self.list_AABBs[AABB_obj_idx].bl.y / self.cellSize)
        upper_x = int(self.list_AABBs[AABB_obj_idx].tr.x / self.cellSize) + 1 if self.list_AABBs[AABB_obj_idx].tr.x % self.cellSize == 0 else int(self.list_AABBs[AABB_obj_idx].tr.x / self.cellSize)
        lower_x = int(self.list_AABBs[AABB_obj_idx].bl.x / self.cellSize) + 1 if self.list_AABBs[AABB_obj_idx].bl.x % self.cellSize == 0 else int(self.list_AABBs[AABB_obj_idx].bl.x / self.cellSize)
        if lower_x == upper_x:
            upper_x += 1
        if lower_y == upper_y:
            upper_y += 1
        for i in range(lower_y,upper_y):
            for j in range(lower_x,upper_x):
                self.list_cells[i][j].appendCell(AABB_obj_idx)
                self.obj_dict[AABB_obj_idx].append([i,j])
                    
    def AABBoutCell(self,AABB_obj_idx):
        for i in self.obj_dict[AABB_obj_idx]:
            self.list_cells[i[0]][i[1]].removeCell(AABB_obj_idx)
        
    def processing(self):
        for i in range(len(self.list_AABBs)):
            self.AABBtoCell(i)
            
        for key,val in self.obj_dict.items():
            for cell in val:
                for obj in self.list_cells[cell[0]][cell[1]].getObj():
                    if (obj == key):
                        continue
                    if (check_collision(self.list_AABBs[key],self.list_AABBs[obj])):
                        self.list_ret.append([key,obj])
            self.AABBoutCell(key)
#             print(self.list_ret, "\n\n\n")
        return set(tuple(x) for x in self.list_ret)
        
        

In [93]:
list_AABBs = []
for i in range(10):
    init_bl_x = np.random.randint(1,25)
    init_bl_y = np.random.randint(1,25)
    list_AABBs.append(AABB(Vec(init_bl_x,init_bl_y),Vec(np.random.randint(init_bl_x,30),np.random.randint(init_bl_y,30))))

for i in list_AABBs:
    print(i.bl.x,i.bl.y,i.tr.x,i.tr.y)
test_grid = Grid(list_AABBs)
output = test_grid.processing()

23 13 28 25
14 18 26 21
12 14 28 22
4 1 13 2
24 7 29 26
19 5 25 27
17 1 27 23
5 22 24 28
19 16 23 22
1 23 25 28


In [95]:
file = open('out.csv', 'w+', newline ='') 
with file:     
    write = csv.writer(file) 
    for i in output:
        write.writerow(i) 